In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from data import get_dichalcogenides_innopolis_202105

In [3]:
structures = get_dichalcogenides_innopolis_202105()

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [4]:
import matplotlib

from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [5]:
import wandb
from wandb.keras import WandbCallback

In [6]:
from sklearn.model_selection import train_test_split

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [7]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [8]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)


In [9]:
config = wandb.config
config.target = "homo"

In [10]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc, npass=5)

In [11]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=True)
model.target_scaler = scaler

In [13]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=100, verbose=1, patience=1000, prev_model="callback/val_mae_00079_0.069863.hdf5")

Epoch 1/100
21/21 [==============================] - 93s 3s/step - loss: 0.0772


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 0.06995, saving model to callback/val_mae_00001_0.069946.hdf5


Epoch 2/100
21/21 [==============================] - 57s 3s/step - loss: 0.0835
Epoch 3/100
21/21 [==============================] - 57s 3s/step - loss: 0.0840
Epoch 4/100
21/21 [==============================] - 57s 3s/step - loss: 0.0827
Epoch 5/100
21/21 [==============================] - 57s 3s/step - loss: 0.0787
Epoch 6/100
21/21 [==============================] - 57s 3s/step - loss: 0.0787
Epoch 7/100
21/21 [==============================] - 57s 3s/step - loss: 0.0776
Epoch 8/100
21/21 [==============================] - 57s 3s/step - loss: 0.0792
Epoch 9/100
21/21 [==============================] - 56s 3s/step - loss: 0.0798
Epoch 10/100
21/21 [==============================] - 57s 3s/step - loss: 0.0784
Epoch 11/100
21/21 [==============================] - 57s 3s/step - loss: 0.0800
Epoch 12/100
21/21 [==============================] - 57s 3s/step - loss: 0.0786
Epoch 13/100
21/21 [==============================] - 57s 3s/step - loss: 0.0781
Epoch 14/100
21/21 [================

INFO:megnet.callbacks:
Epoch 00024: val_mae improved from 0.06995 to 0.06980, saving model to callback/val_mae_00024_0.069799.hdf5


Epoch 25/100
21/21 [==============================] - 57s 3s/step - loss: 0.0778
Epoch 26/100
21/21 [==============================] - 57s 3s/step - loss: 0.0802
Epoch 27/100
21/21 [==============================] - 56s 3s/step - loss: 0.0790
Epoch 28/100
21/21 [==============================] - 57s 3s/step - loss: 0.0790
Epoch 29/100
21/21 [==============================] - 57s 3s/step - loss: 0.0774
Epoch 30/100
21/21 [==============================] - 57s 3s/step - loss: 0.0785
Epoch 31/100
21/21 [==============================] - 57s 3s/step - loss: 0.0765
Epoch 32/100
21/21 [==============================] - 57s 3s/step - loss: 0.0801
Epoch 33/100
21/21 [==============================] - 63s 3s/step - loss: 0.0784
Epoch 34/100
21/21 [==============================] - 89s 4s/step - loss: 0.0773
Epoch 35/100
21/21 [==============================] - 56s 3s/step - loss: 0.0776


...

Epoch 36/100
21/21 [==============================] - 64s 3s/step - loss: 0.0786
Epoch 37/100
21/21 [==============================] - 75s 4s/step - loss: 0.0766
Epoch 38/100
21/21 [==============================] - 81s 4s/step - loss: 0.0794
Epoch 39/100
21/21 [==============================] - 104s 5s/step - loss: 0.0770
Epoch 40/100
21/21 [==============================] - 127s 6s/step - loss: 0.0779
Epoch 41/100
21/21 [==============================] - 195s 9s/step - loss: 0.0771
Epoch 42/100
21/21 [==============================] - 410s 20s/step - loss: 0.0798
Epoch 43/100
21/21 [==============================] - 164s 8s/step - loss: 0.0772


INFO:megnet.callbacks:
Epoch 00043: val_mae improved from 0.06949 to 0.06948, saving model to callback/val_mae_00043_0.069484.hdf5


Epoch 44/100
21/21 [==============================] - 132s 6s/step - loss: 0.0769
Epoch 45/100
21/21 [==============================] - 239s 12s/step - loss: 0.0761
Epoch 46/100
21/21 [==============================] - 240s 11s/step - loss: 0.0780
Epoch 47/100
21/21 [==============================] - 496s 24s/step - loss: 0.0777
Epoch 48/100
21/21 [==============================] - 369s 18s/step - loss: 0.0785
Epoch 49/100
21/21 [==============================] - 589s 29s/step - loss: 0.0773
Epoch 50/100
21/21 [==============================] - 304s 14s/step - loss: 0.0774
Epoch 51/100
21/21 [==============================] - 56s 3s/step - loss: 0.0770
Epoch 52/100
21/21 [==============================] - 60s 3s/step - loss: 0.0769
Epoch 53/100
21/21 [==============================] - 100s 5s/step - loss: 0.0791
Epoch 54/100
21/21 [==============================] - 79s 4s/step - loss: 0.0764


INFO:megnet.callbacks:
Epoch 00054: val_mae improved from 0.06948 to 0.06916, saving model to callback/val_mae_00054_0.069160.hdf5


Epoch 55/100
21/21 [==============================] - 106s 5s/step - loss: 0.0789
Epoch 56/100
21/21 [==============================] - 84s 4s/step - loss: 0.0770
Epoch 57/100
21/21 [==============================] - 57s 3s/step - loss: 0.0759
Epoch 58/100
21/21 [==============================] - 57s 3s/step - loss: 0.0770
Epoch 59/100
21/21 [==============================] - 57s 3s/step - loss: 0.0762
Epoch 60/100
21/21 [==============================] - 56s 3s/step - loss: 0.0791
Epoch 61/100
21/21 [==============================] - 56s 3s/step - loss: 0.0785
Epoch 62/100
21/21 [==============================] - 57s 3s/step - loss: 0.0773
Epoch 63/100
21/21 [==============================] - 57s 3s/step - loss: 0.0779
Epoch 64/100
21/21 [==============================] - 57s 3s/step - loss: 0.0782
Epoch 65/100
21/21 [==============================] - 57s 3s/step - loss: 0.0765
Epoch 66/100
21/21 [==============================] - 57s 3s/step - loss: 0.0792
Epoch 67/100
21/21 [=======